# Tutorial 5 (Enhanced): AutoML AGENT — Classification + Regression

End-to-end optimization with a simulated search strategy. Includes:
- Classification dataset
- Regression dataset
- Search history and leaderboard
- Best config summary and quick inference

In [ ]:
# Install
import sys, subprocess, pkgutil
for p in ['numpy','torch','matplotlib','seaborn','requests','scikit-learn','pandas']:
    if pkgutil.find_loader(p) is None: subprocess.check_call([sys.executable,'-m','pip','install',p])
print('✅ Dependencies ready')

In [ ]:
from tutorial_utils import (
    ping_server,
    ensure_dataset,
    ingest_tensor,
    fetch_dataset,
    summarize_records,
    tensor_addition,
    pretty_json,
)
API = "http://127.0.0.1:7860"
SERVER = ping_server(API)
print(f"📡 Tensorus server available: {SERVER}")

In [ ]:
# Setup
import random, numpy as np, pandas as pd, requests
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
sns.set_theme(style='whitegrid')
API='http://127.0.0.1:7860'
def server_ok():
    try: return requests.get(f'{API}/health', timeout=2).status_code==200
    except: return False
SERVER=server_ok(); print('📡 Tensorus:', '✅ Connected' if SERVER else '⚠️ Demo Mode')

## Part A — Classification

In [ ]:
X,y=make_classification(n_samples=1500, n_features=24, n_informative=10, random_state=7)
Xtr,Xva,ytr,yva=train_test_split(X,y,test_size=0.25,random_state=7)
space={
  'arch':['mlp','deep_mlp','wide_mlp','residual_mlp'],
  'hidden':[(64,),(128,),(256,),(128,64),(256,128,64)],
  'lr':[0.001,0.003,0.01,0.03],
  'bs':[16,32,64,128],
  'drop':[0.0,0.1,0.2,0.3]
}
def score(cfg):
    base=0.72; arch={'mlp':0.0,'deep_mlp':0.05,'wide_mlp':0.03,'residual_mlp':0.08}[cfg['arch']]
    lr_bonus=-abs(cfg['lr']-0.01)*2; depth=len(cfg['hidden'])*0.015; drop=0.02 if 0.1<=cfg['drop']<=0.3 else -0.01
    return float(np.clip(base+arch+lr_bonus+depth+drop+np.random.normal(0,0.02),0.5,0.99))
def rand_cfg():
    return {'arch':random.choice(space['arch']),'hidden':random.choice(space['hidden']),'lr':random.choice(space['lr']),'bs':random.choice(space['bs']),'drop':random.choice(space['drop'])}
def mutate(cfg):
    c=dict(cfg); k=random.choice(list(c.keys()));
    if k in space: c[k]=random.choice(space[k]); return c
    return rand_cfg()
hist=[]; best=None
for t in range(40):
    cfg=rand_cfg() if (best is None or t<5 or np.random.rand()<0.35) else mutate(best['config'])
    sc=score(cfg); rec={'trial':t+1,'score':sc,'config':cfg}; hist.append(rec)
    if (best is None) or sc>best['score']: best=rec
print('Best (classification):', best['score'])
plt.figure(); plt.plot([h['score'] for h in hist],'-o'); plt.title('Classification Optimization'); plt.show()
pd.DataFrame(sorted(hist,key=lambda x:x['score'],reverse=True)[:5])

## Part B — Regression

In [ ]:
X,y=make_regression(n_samples=1200, n_features=16, noise=7.5, random_state=11)
Xtr,Xva,ytr,yva=train_test_split(X,y,test_size=0.25,random_state=11)
space_r={'hidden':[(64,),(128,64),(256,128,64)],'lr':[0.001,0.003,0.01,0.03],'bs':[16,32,64,128],'drop':[0.0,0.1,0.2]}
def rscore(cfg):
    base=0.65; depth=len(cfg['hidden'])*0.02; lr=-abs(cfg['lr']-0.003)*1.5; drop=(0.02 if cfg['drop']<=0.2 else -0.01)
    return float(np.clip(base+depth+lr+drop+np.random.normal(0,0.02),0.45,0.97))
def rrand(): return {'hidden':random.choice(space_r['hidden']),'lr':random.choice(space_r['lr']),'bs':random.choice(space_r['bs']),'drop':random.choice(space_r['drop'])}
def rmut(cfg):
    c=dict(cfg); k=random.choice(list(c.keys())); c[k]=random.choice(space_r[k]); return c
hist_r=[]; best_r=None
for t in range(35):
    cfg=rrand() if (best_r is None or t<5 or np.random.rand()<0.35) else rmut(best_r['config'])
    sc=rscore(cfg); rec={'trial':t+1,'score':sc,'config':cfg}; hist_r.append(rec)
    if (best_r is None) or sc>best_r['score']: best_r=rec
print('Best (regression):', best_r['score'])
plt.figure(); plt.plot([h['score'] for h in hist_r],'-o',color='orange'); plt.title('Regression Optimization'); plt.show()
pd.DataFrame(sorted(hist_r,key=lambda x:x['score'],reverse=True)[:5])

## Quick Inference (Simulated)

In [ ]:
val_scores_cls = np.random.normal(loc=best['score'], scale=0.01, size=10)
val_scores_reg = np.random.normal(loc=best_r['score'], scale=0.01, size=10)
print('Validation scores (classification):', np.round(val_scores_cls,4))
print('Validation scores (regression):', np.round(val_scores_reg,4))